In [32]:
from kubernetes import client, config, watch
import json
import base64

In [33]:
namespace = 'volume-project'
volume_name = 'job1-volume-claim'
secret_name = 'bespin-dukeds-agent-creds'
agent_key = 'TODO'
user_key = 'TODO'
ddsclient_secret_config = json.dumps({"agent_key": agent_key, "user_key": user_key})

In [34]:
config.load_kube_config()

In [3]:
core_api_instance = client.CoreV1Api()
batch_api_instance = client.BatchV1Api()

In [5]:
def create_persistent_volume_claim(client, api_instance, namespace, name, storage_size_in_g):
    pvc = client.V1PersistentVolumeClaim()
    pvc.api_version = "v1"
    pvc.kind = "PersistentVolumeClaim"
    pvc.metadata = client.V1ObjectMeta(name=name)
    pvc.spec = client.V1PersistentVolumeClaimSpec(access_modes=["ReadWriteMany"],
        resources=client.V1ResourceRequirements(requests={"storage" : "{}Gi".format(storage_size_in_g)}),
        storage_class_name="glusterfs-storage")
    api_instance.create_namespaced_persistent_volume_claim(namespace, pvc)

In [6]:
def delete_persistent_volume_claim(client, api_instance, namespace, name):
    body = client.V1DeleteOptions()
    api_instance.delete_namespaced_persistent_volume_claim(name, namespace, body)

In [7]:
def create_secret(client, api_instance, namespace, name, string_data):
    body = client.V1Secret()
    api_instance.create_namespaced_secret(namespace=namespace, body=client.V1Secret(
        string_data=string_data,
        metadata={
          'name': name
        }
      )
    )

In [8]:
def create_job(client, api_instance, namespace, job_name, secret_name, image_name, volume_claim_name):
    job_metadata = client.V1ObjectMeta(name=job_name)
    container = client.V1Container(
        image=image_name,
        name="stage-data",
        working_dir="/goodstuff",
        command=["ddsclient","download", "-p", "esd", "--include", "somefile.txt"],
        env=[
          client.V1EnvVar(
              name="DDSCLIENT_CONF",
              value="/etc/ddsclient/config"
          )  
        ],
        volume_mounts=[
            client.V1VolumeMount(
                name="ddsclient",
                mount_path="/etc/ddsclient"
            ),
            client.V1VolumeMount(
                name="datadest",
                mount_path="/goodstuff"
            )            
        ]
    )
    spec = client.V1JobSpec(
        template=client.V1PodTemplateSpec(
            metadata=client.V1ObjectMeta(name="job1spec"),
            spec=client.V1PodSpec(
                containers=[container], 
                volumes=[client.V1Volume(
                    name="ddsclient", 
                    secret=client.V1SecretVolumeSource(
                        secret_name=secret_name
                    )
                ),client.V1Volume(
                    name="datadest",
                    persistent_volume_claim=client.V1PersistentVolumeClaimVolumeSource(
                        claim_name=volume_claim_name
                    )
                )],
                restart_policy="Never"
            )
        )
    )
    body = client.V1Job(
        metadata=job_metadata,
        spec=spec)
    api_instance.create_namespaced_job(namespace, body)

In [36]:
def delete_job(client, api_instance, namespace, name):
    # propagation_policy controls deleting pods
    api_instance.delete_namespaced_job(name, namespace, 
                                       body=client.V1DeleteOptions(propagation_policy='Background'))

In [10]:
def delete_secret(client, api_instance, namespace, name):
    api_instance.delete_namespaced_secret(name, namespace, body=client.V1DeleteOptions())

In [46]:
def watch_jobs_once(client, api_instance, namespace):
    job_list = api_instance.list_namespaced_job(namespace)
    print("JOBS")
    for job in job_list.items:
        print("name: {} active:{} succeeded:{} failed: {}".format(
            job.metadata.name, job.status.active, job.status.succeeded, job.status.failed))

def watch_jobs_loop(client, api_instance, namespace):
    count = 1
    w = watch.Watch()
    for event in w.stream(api_instance.list_namespaced_job, namespace, _request_timeout=1):
        job = event['object']
        print("{} name: {} active:{} succeeded:{} failed: {}".format(event['type'],
            job.metadata.name, job.status.active, job.status.succeeded, job.status.failed))
        count -= 1
        if not count:
            w.stop()

In [38]:
job_name = "job10"
create_persistent_volume_claim(client, core_api_instance, namespace, volume_name, storage_size_in_g=2)
create_secret(client, core_api_instance, namespace, secret_name, {"config": ddsclient_secret_config})
create_job(client, batch_api_instance, namespace, job_name, secret_name, image_name="jbradley/dukedsclient",
           volume_claim_name=volume_name)

/Users/jpb67/Documents/work/k8vol/venv/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/jpb67/Documents/work/k8vol/venv/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/jpb67/Documents/work/k8vol/venv/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [47]:
#watch_jobs_once(client, batch_api_instance, namespace)
watch_jobs_loop(client, batch_api_instance, namespace)

ADDED name: job10 active:None succeeded:1 failed: None


/Users/jpb67/Documents/work/k8vol/venv/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [48]:
delete_job(client, batch_api_instance, namespace, job_name)
delete_secret(client, core_api_instance, namespace, secret_name)
delete_persistent_volume_claim(client, core_api_instance, namespace, volume_name)

/Users/jpb67/Documents/work/k8vol/venv/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/jpb67/Documents/work/k8vol/venv/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/Users/jpb67/Documents/work/k8vol/venv/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
